# Introduction to Python using Turing patterns

## Variables
Variables are symbolics names for information.

For example a 

# Turing patterns
You can find here a simple, naive implementation of the Turing patterns.

There are few cosmetic deviations from a normal implementation (especially the variable dt).

For info: [@guignardlab](https://twitter.com/guignardlab)

In [ ]:
import numpy as np
from colorsys import rgb_to_hsv, hsv_to_rgb
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import HTML
from scipy.signal import convolve2d
from scipy.stats import logistic
%matplotlib inline

## CenTuri "colormap"
RGB = np.array([0, 137, 179])/255
HSV = rgb_to_hsv(*RGB)
colors = [hsv_to_rgb(HSV[0], 0, HSV[1]), hsv_to_rgb(HSV[0], 1, HSV[1])]
cmap = LinearSegmentedColormap.from_list("CenTuri_cmap", colors)

# Useful functions
def norm_array(array, mask, th=None, func=None):
    """
    Normalizing an array between 0 and 1
    
    Parameters
    ----------
    array : ndarray of shape 2 or 3
        The array to be normalize. If the shape of the
        array is 3, each 2D arrays will be normalized
        independently along the axis 0
    mask : ndarray of with the same shape of array
        Puts to 0 values that are outside of the mask
    th : float
        any value bellow the threshold are put to 0
    func : function
        a function that will be applied to the normalized
        array if th is None
    Returns
    -------
    out : ndarray
        The normalized array
    """
    if len(array.shape)==2:
        vals_to_norm = array[mask!=0]
        out = (array-vals_to_norm.min())/(vals_to_norm.max() - vals_to_norm.min())
        out[mask==0] = 0
    else:
        vals_to_norm = array[mask!=0]
        out = (array-vals_to_norm.min(axis=0))/(vals_to_norm.max(axis=0) - vals_to_norm.min(axis=0))
        out[mask==0] = 0
    if th:
        out[out<th] = 0
    elif func:
        out = func(out)
    return out

def outer_circle(XY, centre, r):
    """
    From an np.mgrid `XY`, builds a mask that is True within
    the circle centered in `centre` and of radius `r`

    Parameters
    ----------
    XY : 2D mgrid
        The support of the mask
    centre: tuple of floats
        The coordinates of the centre of circle
    r: float
        Radius of the circle
    
    Returns
    -------
    out : ndarray of the same shape of XY
        The mask
    """
    return (XY[0]-centre[0])**2 + (XY[1]-centre[1])**2 < r**2

We have an activator $A$ and an inhibitor $I$ which interact as follow:

$A \rightarrow A$

$A \rightarrow I$

$I \dashv A$

Resulting in the following equations:

$\frac{\delta A}{\delta t} = k_{AA} A - k_{IA} I$

$\frac{\delta I}{\delta t} = k_{AI} A - k_{II} I$

In [ ]:
# Functions for the evolution of A and I
dA_dt = lambda AI: (kAA * AI[0] - kIA * AI[1])
dI_dt = lambda AI: (kAI * AI[0] - kII * AI[1])

def dAI_dt(AI):
    return dA_dt(AI), dI_dt(AI)

The antagonist (resp. inhibitor) diffuses according to a diffusion coefficient $\mu$ (resp. $\nu$) to the 4 neighboring cells. The diffusion is modeled as follow:

$\frac{\delta A_{x, y}}{\delta x \delta y} = \mu(A_{x+\delta x, y} + A_{x-\delta x, y} + A_{x, y+\delta y} + A_{x, y-\delta y} - 4A_{x, y})$

$\frac{\delta I_{x, y}}{\delta x \delta y} = \nu(I_{x+\delta x, y} + I_{x-\delta x, y} + I_{x, y+\delta y} + I_{x, y-\delta y} - 4I_{x, y})$

In [ ]:
def diffusion(array, weights):
    """
    Computes the diffusion in 2D given a set of weights.
    The weights are useful for the average at the borders
    of the region considered.
    If this is the region considered:
    [[ 0 0 0 0 0 0 ]
     [ 0 0 1 1 0 0 ]
     [ 0 1 1 1 1 0 ]
     [ 0 1 1 1 1 0 ]
     [ 0 0 1 1 0 0 ]
     [ 0 0 0 0 0 0 ]]
    the weights should be the number of neighbors
    within the mask:
    [[ 0 0 0 0 0 0 ]
     [ 0 0 2 2 0 0 ]
     [ 0 2 4 4 2 0 ]
     [ 0 2 4 4 2 0 ]
     [ 0 0 2 2 0 0 ]
     [ 0 0 0 0 0 0 ]]
    this can be obtained by convolving
    the mask with the following kernel:
    [[ 0 1 0 ]
     [ 1 0 1 ]
     [ 0 1 0 ]]
    
    Parameters
    ----------
    array : ndarray of floats shape NxM
        The array with the initial concentrations
    weights : ndarray of floats of shape NxM
        The weights for the diffusion
    
    Returns
    -------
    out : ndarray of shape NxM
        the initial array after diffusion
        (without taking into account the
        diffusion coefficients)
    """
    kernel = [[0, 1, 0],
              [1, 0, 1],
              [0, 1, 0]]
    out = convolve2d(array, kernel, boundary='symm', mode='same')
    out -= array * weights
    return out

Then we combine the activation/inhibition and diffusion and we get:

$\frac{\delta A_{x, y}}{\delta t\delta x \delta y} = k_{f} A - k_{r} I + \mu(A_{x+\delta x, y} + A_{x-\delta x, y} + A_{x, y+\delta y} + A_{x, y-\delta y} - 4A_{x, y})$

$\frac{\delta I_{x, y}}{\delta t\delta x \delta y} = k_{e} A - k_{d} I + \nu(I_{x+\delta x, y} + I_{x-\delta x, y} + I_{x, y+\delta y} + I_{x, y-\delta y} - 4I_{x, y})$

In [ ]:
def dAI_dtxy(A, I, dt, dx, dy, mu, nu, weights):
    """
    Integrating activation, inhibition and diffusion
    
    Parameters
    ----------
    A : ndarray of floats of shape NxM
        The array containing the initial
        concentration of activators
    I : ndarray of floats of shape NxM
        The array containing the initial
        concentration of inhibitors
    dt : float
        time increment
    dx : float
        size of a pixel in `A` and `I` in µm
        along the `x` dimension
    dy : float
        size of a pixel in `A` and `I` in µm
        along the `y` dimension
    mu : float
        diffusion coefficient for the activator
    nu : float
        diffusion coefficient for the inhibitor
    weights : ndarray of shape NxM
        weights for the diffusion average related
        to the mask considered
    
    Returns
    -------
    A : ndarray of floats of shape NxM
        The array containing the new
        concentration of activators after
        a time dt
    I : ndarray of floats of shape NxM
        The array containing the new
        concentration of inhibitors after
        a time dt
    """
    diffusion_A = diffusion(A, weights)/(dx*dy)
    diffusion_I = diffusion(I, weights)/(dx*dy)
    new_A = A + dt*(dA_dt([A, I]) + mu*diffusion_A)
    new_I = I + dt*(dI_dt([A, I]) + nu*diffusion_I)
    A = new_A
    I = new_I
    A[mask==0] = 0
    I[mask==0] = 0
    return A, I

Carefully chosen parameters, thanks to Alice Gros in [@Equipe_lenne](https://twitter.com/Equipe_lenne)

In [ ]:
kAA, kIA, kAI, kII = 10, 12, 31, 20
mu = 0.001
nu = 0.025

# voxel size in the x and y dimensions
dx = dy = 0.02

# step size of time in seconds
dt = 0.000001
# because the timestep has to be smaller
# at the begining we use a variable dt
dt_final = 0.001
variable_dt = np.linspace(dt, dt_final, n)

# Number of iterations
n = 20000
# Number of timepoints to save
nb_times_im = 200

Random initialisation of the circle

Computation of the weight mask

Initialisation of the final array which will contain our data

In [ ]:
# size of the grid onto which the model is run
grid_size = 100
# centre and radius of the circle onto wich
# the model is run
circle_center = [49, 49]
radius = 45

# randomizing the initial state
# change the seed from 1 to any other value
# to observe different results
generator = np.random.default_rng(1)
init_A = generator.uniform(size=(grid_size, grid_size))
init_I = generator.uniform(size=(grid_size, grid_size))

# building the mask to get a circle
# together with the weights related
# to this specific mask.
mask = (outer_circle(np.mgrid[:grid_size, :grid_size], circle_center, r=radius)).astype(np.uint8)
kernel = [[0, 1, 0],
          [1, 0, 1],
          [0, 1, 0]]
weights = convolve2d(mask, kernel, boundary='symm', mode='same')

# Masking the initial position and
# copying it for safety
init_A[mask==0] = 0
init_I[mask==0] = 0
A = init_A.copy()
I = init_I.copy()

# ndarray in which will be stored the results
final_A = np.zeros(A.shape + (nb_times_im+1,))

Running the simulation

In [ ]:
for i in range(n):
    if variable_dt is not None:
        dt = variable_dt[i]
    if i%(n//nb_times_im)==0:
        final_A[...,int(i//(n/nb_times_im))] = A
    A, I = dAI_dtxy(A, I, dt, dx, dx, mu, nu, weights)
    
final_A[...,-1] = A

In [ ]:
# Normalizing the final array using a logistic function
# centered in 0.5 to have values between 0 and 1 but
# a sharper slope between low and high expression
normed_array = norm_array(final_A, mask, func=lambda x:logistic.cdf(x, .5, .05))

Plotting the results

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.axis('off')
im = ax.imshow(init_A, cmap=cmap)
fig.tight_layout()

def init():
    im.set_data(init_A)
    return(im,)

def animate(i):
    im.set_data(normed_array[...,i])
    return(im,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=nb_times_im, interval=25, 
                               blit=True)

HTML(anim.to_jshtml())

In [ ]:
anim.save('turing.gif', dpi=50)